## 1. Import libraries and dependencies

In [1]:
from src.preprocess.preprocess import Preprocess
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import time
from src.preprocess.pipeline import Pipeline

/home/hubert/Workspace/Python/ML Project/languages
{'cs': '/home/hubert/Workspace/Python/ML Project/languages/cs', 'lt': '/home/hubert/Workspace/Python/ML Project/languages/lt', 'pl': '/home/hubert/Workspace/Python/ML Project/languages/pl'}
{'cs': '/home/hubert/Workspace/Python/ML Project/languages/cs/validated.tsv', 'lt': '/home/hubert/Workspace/Python/ML Project/languages/lt/validated.tsv', 'pl': '/home/hubert/Workspace/Python/ML Project/languages/pl/validated.tsv'}
{'cs': '/home/hubert/Workspace/Python/ML Project/languages/cs/clip_durations.tsv', 'lt': '/home/hubert/Workspace/Python/ML Project/languages/lt/clip_durations.tsv', 'pl': '/home/hubert/Workspace/Python/ML Project/languages/pl/clip_durations.tsv'}


2024-05-23 21:36:26.215712: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-23 21:36:26.218424: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-23 21:36:26.255548: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-23 21:36:26.906417: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## 2. Preprocess audio

In [2]:
ORIGIN_SAMPLE_RATE = 48_000
FINAL_SAMPLE_RATE = 16_000
MAX_CLIENT_ID_AMOUNT = 2500
MIN_CLIP_DURATION_MS = 2000
SET_SIZE = 20_000
BATCH_SIZE = 64

In [3]:
start = time.time()
Preprocess.initialize(
    batch_size=BATCH_SIZE,
    max_client_id_amount=MAX_CLIENT_ID_AMOUNT,
    min_clip_duration_ms=MIN_CLIP_DURATION_MS,
    set_size=SET_SIZE,
    origin_sample_rate=ORIGIN_SAMPLE_RATE,
    final_sample_rate=FINAL_SAMPLE_RATE,
)
Preprocess.preprocess_probes()
end = time.time()
print(f'Time taken: {end - start}')

PROCESSED LANGUAGES: cs
---AUDIO META INFO DONE cs---
---SET SPLIT DONE 0 cs---
---SET SPLIT DONE 1 cs---
PROCESSED LANGUAGES: lt
---AUDIO META INFO DONE lt---
---SET SPLIT DONE 0 lt---
---SET SPLIT DONE 1 lt---
PROCESSED LANGUAGES: pl
---AUDIO META INFO DONE pl---
---SET SPLIT DONE 0 pl---
---SET SPLIT DONE 1 pl---
Time taken: 4.957049369812012


In [4]:
print(f"Original sample rate: {Preprocess.ORIGIN_SAMPLE_RATE}")
print(f"Final sample rate: {Preprocess.SAMPLE_RATE}")
print(f"Max client id amount: {Preprocess.MAX_CLIENT_ID_AMOUNT}")
print(f"Min clip duration in milliseconds: {Preprocess.MIN_CLIP_DURATION_MS}")
print(f"Set size: {Preprocess.SET_SIZE}")
print(f"One language set size: {Preprocess.ONE_LANG_SET_SIZE}")

print("-"*25 + "SETS SIZES" + "-"*25)
print(f"Training set size: {Preprocess.SET_HPARAMS.train_size}")
print(f"Val set size: {Preprocess.SET_HPARAMS.val_size}")
print(f"Test set size: {Preprocess.SET_HPARAMS.test_size}")
print(f"Is test + val + train is equal one lang set size: "
      f"\n{Preprocess.SET_HPARAMS.test_size + Preprocess.SET_HPARAMS.val_size + Preprocess.SET_HPARAMS.train_size} <= {Preprocess.ONE_LANG_SET_SIZE}")

Original sample rate: 48000
Final sample rate: 16000
Max client id amount: 2500
Min clip duration in milliseconds: 2000
Set size: 20000
One language set size: 6666
-------------------------SETS SIZES-------------------------
Training set size: 3999
Val set size: 1332
Test set size: 1332
Is test + val + train is equal one lang set size: 
6663 <= 6666


In [5]:
train = Preprocess.TRAIN_FILENAMES
val = Preprocess.VAL_FILENAMES
test = Preprocess.TEST_FILENAMES

In [6]:
train_dataset = Pipeline.create_pipeline(train, is_train=True)
val_dataset = Pipeline.create_pipeline(val)
test_dataset = Pipeline.create_pipeline(test)

2024-05-23 21:36:33.747811: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-23 21:36:33.748179: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-05-23 21:36:33.860577: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA


ADD ZEROS
CUT AUDIO
AUGUMENT
ADD ZEROS
CUT AUDIO
ADD ZEROS
CUT AUDIO


In [7]:
# counter = 0
# start = time.time()
# for batch, label in train_dataset:
#     print(batch.shape)
# 
# end = time.time()
# print(f"Train it time: {end - start}")
# print(counter)


In [8]:
# counter = 0
# start = time.time()
# for batch, label in val_dataset:
#     print(batch.shape)
#     for sample in batch:
#         print(sample)
#         counter += 1
# end = time.time()
# print(f"Val it time: {end - start}")
# print(counter)

In [9]:
# counter = 0
# start = time.time()
# for batch, label in test_dataset:
#     print(batch.shape)
#     for sample in batch:
#         print(sample)
#         counter += 1
# end = time.time()
# print(f"Test it time: {end - start}")
# print(counter)

In [10]:
test_dataset.element_spec

(TensorSpec(shape=(64, 991, 257, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(64, 3), dtype=tf.int32, name=None))

In [11]:
val_dataset.element_spec

(TensorSpec(shape=(64, 991, 257, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(64, 3), dtype=tf.int32, name=None))

In [12]:
train_dataset.element_spec

(TensorSpec(shape=(64, 991, 257, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(64, 3), dtype=tf.int32, name=None))

## 4. Create model

In [13]:
input_shape = train_dataset.as_numpy_iterator().next()[0].shape[1:]
input_shape

2024-05-23 21:36:36.784651: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


(991, 257, 1)

In [14]:
model = keras.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv2D(8, (5, 3), activation='relu', padding='valid'),
    layers.MaxPooling2D((3, 2)),
    layers.Conv2D(16, (5, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((3, 2)),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((3, 3)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((3, 3)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 987, 255, 8)    │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 329, 127, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 329, 127, 16)   │         1,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 109, 63, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 109, 63, 32)    │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 36, 21, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 36, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 12, 7, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 5376)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,753,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,991,987 (11.41 MB)

 Trainable params: 2,991,987 (11.41 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset, batch_size=BATCH_SIZE)

Epoch 1/10
 15/187 ━━━━━━━━━━━━━━━━━━━━ 6:21 2s/step - accuracy: 0.3611 - loss: 1.1051

In [ ]:
# test_loss, test_acc = model.evaluate(Preprocess.test_dataset, verbose=2)
# print(f'\nTest accuracy: {test_acc}')

In [ ]:
# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
# plt.legend(loc='lower right')
# plt.show()
